In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

24628


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
24623,2021-06-18,47,沖縄県,86,19943,0,167
24624,2021-06-19,47,沖縄県,97,20040,0,167
24625,2021-06-20,47,沖縄県,61,20101,0,167
24626,2021-06-21,47,沖縄県,33,20134,0,167
24627,2021-06-22,47,沖縄県,98,20232,4,171


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,11528,9067,148
43,香川県,19388,2083,30
44,高知県,20436,1749,24
45,鳥取県,16244,466,2
46,鹿児島県,24104,3641,38


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
24621,2021-06-16,47,沖縄県,115,19760,0,163,848.0
24622,2021-06-17,47,沖縄県,97,19857,4,167,779.0
24623,2021-06-18,47,沖縄県,86,19943,0,167,720.0
24624,2021-06-19,47,沖縄県,97,20040,0,167,660.0
24625,2021-06-20,47,沖縄県,61,20101,0,167,617.0
24626,2021-06-21,47,沖縄県,33,20134,0,167,596.0
24627,2021-06-22,47,沖縄県,98,20232,4,171,587.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
24618,2021-06-13,47,沖縄県,104,19484,0,159,1009.0,-0.337580,-0.033687,-0.431694
24619,2021-06-14,47,沖縄県,54,19538,4,163,959.0,-0.480769,-0.040698,-0.480769
24620,2021-06-15,47,沖縄県,107,19645,0,163,907.0,0.981481,0.023964,-0.327044
24621,2021-06-16,47,沖縄県,115,19760,0,163,848.0,0.074766,0.021168,-0.339080
24622,2021-06-17,47,沖縄県,97,19857,4,167,779.0,-0.156522,0.005376,-0.415663
24623,2021-06-18,47,沖縄県,86,19943,0,167,720.0,-0.113402,0.007248,-0.406897
24624,2021-06-19,47,沖縄県,97,20040,0,167,660.0,0.127907,0.013697,-0.382166
24625,2021-06-20,47,沖縄県,61,20101,0,167,617.0,-0.371134,0.008904,-0.413462
24626,2021-06-21,47,沖縄県,33,20134,0,167,596.0,-0.459016,0.012012,-0.388889
24627,2021-06-22,47,沖縄県,98,20232,4,171,587.0,1.969697,0.153185,-0.084112


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
523,2021-06-22,北海道,29,12,428.0,-0.216216,-0.135670,-0.666667
1047,2021-06-22,青森県,0,0,2.0,NaN,NaN,-1.000000
1571,2021-06-22,岩手県,5,0,32.0,inf,NaN,-0.285714
2095,2021-06-22,宮城県,7,0,30.0,2.500000,0.259524,-0.300000
2619,2021-06-22,秋田県,9,0,21.0,0.285714,NaN,8.000000


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,12576,5192,110,2021-06-22,2,0,40.0,-0.600000,0.251020,-0.500000
1,京都府,13624,16478,236,2021-06-22,12,1,103.0,11.000000,1.432446,-0.333333
2,佐賀県,21484,2546,24,2021-06-22,1,0,7.0,inf,NaN,inf
3,兵庫県,14672,40759,1273,2021-06-22,27,1,187.0,0.928571,0.010216,-0.386364
4,北海道,524,41061,1358,2021-06-22,29,12,428.0,-0.216216,-0.135670,-0.666667


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,12576,5192,110,2021-06-22,2,0,40.0,-0.600000,0.251020,-0.500000,1813859
1,京都府,13624,16478,236,2021-06-22,12,1,103.0,11.000000,1.432446,-0.333333,2545899
2,佐賀県,21484,2546,24,2021-06-22,1,0,7.0,inf,NaN,inf,823810
3,兵庫県,14672,40759,1273,2021-06-22,27,1,187.0,0.928571,0.010216,-0.386364,5549568
4,北海道,524,41061,1358,2021-06-22,29,12,428.0,-0.216216,-0.135670,-0.666667,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,524,41061,1358,2021-06-22,29,12,428.0,-0.216216,-0.135670,-0.666667,5267762,77.947713,779.477129,8.124893
41,青森県,1048,2453,31,2021-06-22,0,0,2.0,NaN,NaN,-1.000000,1275783,19.227408,192.274078,0.156766
19,岩手県,1572,1638,46,2021-06-22,5,0,32.0,inf,NaN,-0.285714,1235517,13.257608,132.576079,2.590009
11,宮城県,2096,9075,86,2021-06-22,7,0,30.0,2.500000,0.259524,-0.300000,2292385,39.587591,395.875911,1.308681
36,秋田県,2620,785,20,2021-06-22,9,0,21.0,0.285714,NaN,8.000000,985416,7.966179,79.661788,2.131080
15,山形県,3144,2020,47,2021-06-22,0,0,8.0,NaN,NaN,-1.000000,1082296,18.664025,186.640254,0.739169
35,福島県,3668,4791,161,2021-06-22,6,0,70.0,-0.400000,0.329659,1.000000,1881981,25.457218,254.572177,3.719485
38,茨城県,4192,10295,162,2021-06-22,19,1,133.0,2.800000,0.395172,0.266667,2921436,35.239519,352.395192,4.552556
27,栃木県,4716,6835,81,2021-06-22,29,0,122.0,0.933333,0.148243,0.160000,1965516,34.774583,347.745834,6.207021
37,群馬県,5240,8010,151,2021-06-22,6,0,22.0,5.000000,0.607143,0.000000,1969439,40.671481,406.714806,1.117069


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      16687.127660
std       30591.646471
min         466.000000
25%        2499.500000
50%        4985.000000
75%       10856.500000
max      169756.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,6812,169756,2203,2021-06-22,435,5,2841.0,0.843220,0.098217,0.290801,13834925,122.701063,1227.010627,20.534987
1,大阪府,14148,102589,2609,2021-06-22,107,4,648.0,1.547619,0.142413,-0.027273,8849635,115.924555,1159.245551,7.322336
2,神奈川県,7336,65761,939,2021-06-22,163,2,1267.0,0.207407,0.023265,0.018750,9209442,71.406063,714.060635,13.757620
3,愛知県,12052,50644,943,2021-06-22,96,4,619.0,1.742857,0.148018,-0.020408,7575530,66.852088,668.520882,8.171045
4,埼玉県,5764,45674,827,2021-06-22,93,1,526.0,0.754717,0.085217,0.328571,7390054,61.804690,618.046905,7.117675
5,北海道,524,41061,1358,2021-06-22,29,12,428.0,-0.216216,-0.135670,-0.666667,5267762,77.947713,779.477129,8.124893
6,兵庫県,14672,40759,1273,2021-06-22,27,1,187.0,0.928571,0.010216,-0.386364,5549568,73.445356,734.453565,3.369632
7,千葉県,6288,39317,701,2021-06-22,74,0,753.0,-0.129412,-0.008162,-0.139535,6319772,62.212687,622.126874,11.914987
8,福岡県,20960,35274,513,2021-06-22,36,1,243.0,1.117647,0.137932,0.028571,5129841,68.762365,687.623651,4.736989
9,沖縄県,24628,20232,171,2021-06-22,98,4,587.0,1.969697,0.153185,-0.084112,1481547,136.559961,1365.599606,39.620748


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      40.257660
std       29.188720
min        7.966179
25%       22.298950
50%       30.280683
75%       52.409418
max      136.559961
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,24628,20232,171,2021-06-22,98,4,587.0,1.969697,0.153185,-0.084112,1481547,136.559961,1365.599606,39.620748
26,東京都,6812,169756,2203,2021-06-22,435,5,2841.0,0.843220,0.098217,0.290801,13834925,122.701063,1227.010627,20.534987
16,山梨県,9956,2019,20,2021-06-22,14,0,157.0,-0.416667,0.265909,0.000000,826579,24.425977,244.259774,18.993950
32,神奈川県,7336,65761,939,2021-06-22,163,2,1267.0,0.207407,0.023265,0.018750,9209442,71.406063,714.060635,13.757620
5,千葉県,6288,39317,701,2021-06-22,74,0,753.0,-0.129412,-0.008162,-0.139535,6319772,62.212687,622.126874,11.914987
33,福井県,9432,1155,34,2021-06-22,17,0,86.0,-0.150000,0.407576,3.250000,780053,14.806686,148.066862,11.024892
44,高知県,20436,1749,24,2021-06-22,16,1,66.0,1.000000,0.158333,0.000000,709230,24.660547,246.605474,9.305867
24,愛知県,12052,50644,943,2021-06-22,96,4,619.0,1.742857,0.148018,-0.020408,7575530,66.852088,668.520882,8.171045
4,北海道,524,41061,1358,2021-06-22,29,12,428.0,-0.216216,-0.135670,-0.666667,5267762,77.947713,779.477129,8.124893
9,大阪府,14148,102589,2609,2021-06-22,107,4,648.0,1.547619,0.142413,-0.027273,8849635,115.924555,1159.245551,7.322336


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    32.000000
mean      0.292123
std       0.391352
min      -0.135670
25%       0.094967
50%       0.168831
75%       0.346037
max       1.773139
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
29,滋賀県,13100,5509,89,2021-06-22,13,0,79.0,12.000000,1.773139,-0.071429,1420948,38.769892,387.698916,5.559669
1,京都府,13624,16478,236,2021-06-22,12,1,103.0,11.000000,1.432446,-0.333333,2545899,64.723699,647.236988,4.045722
30,熊本県,22532,6450,113,2021-06-22,11,0,39.0,2.666667,0.639456,0.571429,1769880,36.443149,364.431487,2.203539
37,群馬県,5240,8010,151,2021-06-22,6,0,22.0,5.000000,0.607143,0.000000,1969439,40.671481,406.714806,1.117069
31,石川県,8908,3920,114,2021-06-22,6,0,37.0,2.000000,0.515986,1.000000,1139612,34.397672,343.976722,3.246719
10,奈良県,15196,8058,126,2021-06-22,23,0,86.0,3.600000,0.421760,0.277778,1353837,59.519721,595.197206,6.352316
33,福井県,9432,1155,34,2021-06-22,17,0,86.0,-0.150000,0.407576,3.250000,780053,14.806686,148.066862,11.024892
38,茨城県,4192,10295,162,2021-06-22,19,1,133.0,2.800000,0.395172,0.266667,2921436,35.239519,352.395192,4.552556
35,福島県,3668,4791,161,2021-06-22,6,0,70.0,-0.400000,0.329659,1.000000,1881981,25.457218,254.572177,3.719485
17,岐阜県,11004,9207,185,2021-06-22,18,2,97.0,2.600000,0.296939,0.285714,2032490,45.299116,452.991159,4.772471


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,524,41061,1358,2021-06-22,29,12,428.0,-0.216216,-0.135670,-0.666667,5267762,77.947713,779.477129,8.124893
18,岡山県,17292,7581,124,2021-06-22,1,0,25.0,0.000000,-0.045238,-0.800000,1903627,39.823978,398.239781,1.313282
5,千葉県,6288,39317,701,2021-06-22,74,0,753.0,-0.129412,-0.008162,-0.139535,6319772,62.212687,622.126874,11.914987
42,静岡県,11528,9067,148,2021-06-22,18,1,173.0,0.200000,-0.006504,-0.217391,3708556,24.448869,244.488690,4.664888
3,兵庫県,14672,40759,1273,2021-06-22,27,1,187.0,0.928571,0.010216,-0.386364,5549568,73.445356,734.453565,3.369632
32,神奈川県,7336,65761,939,2021-06-22,163,2,1267.0,0.207407,0.023265,0.018750,9209442,71.406063,714.060635,13.757620
21,広島県,17816,11418,168,2021-06-22,13,1,149.0,-0.235294,0.055905,-0.071429,2826858,40.391134,403.911339,5.270870
7,埼玉県,5764,45674,827,2021-06-22,93,1,526.0,0.754717,0.085217,0.328571,7390054,61.804690,618.046905,7.117675
26,東京都,6812,169756,2203,2021-06-22,435,5,2841.0,0.843220,0.098217,0.290801,13834925,122.701063,1227.010627,20.534987
40,長野県,10480,4985,91,2021-06-22,7,0,45.0,0.000000,0.111224,0.750000,2087307,23.882448,238.824476,2.155888


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   466.    21627.25  42788.5   63949.75  85111.   106272.25 127433.5
 148594.75 169756.  ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([37.,  3.,  5.,  0.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 9., 23.,  4.,  8.,  0.,  1.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="和歌山県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
15710,2021-06-13,30,和歌山県,1,2652,0,47,15.0,-0.666667,0.428571,-0.500000
15711,2021-06-14,30,和歌山県,0,2652,1,48,14.0,-1.000000,0.357143,-1.000000
15712,2021-06-15,30,和歌山県,2,2654,0,48,15.0,inf,NaN,1.000000
15713,2021-06-16,30,和歌山県,0,2654,0,48,13.0,-1.000000,NaN,-1.000000
15714,2021-06-17,30,和歌山県,1,2655,0,48,8.0,inf,NaN,-0.833333
15715,2021-06-18,30,和歌山県,1,2656,0,48,8.0,0.000000,NaN,0.000000
15716,2021-06-19,30,和歌山県,1,2657,0,48,6.0,0.000000,NaN,-0.666667
15717,2021-06-20,30,和歌山県,3,2660,0,48,8.0,2.000000,NaN,2.000000
15718,2021-06-21,30,和歌山県,0,2660,0,48,8.0,-1.000000,NaN,NaN
15719,2021-06-22,30,和歌山県,3,2663,1,49,9.0,inf,NaN,0.500000
